In [9]:
from ortools.sat.python import cp_model

In [10]:
# Data 
frequencies = [15, 8, 20]
volumes = [2, 1, 3]
distances = [1, 2, 3]

num_products = len(frequencies)
num_slots = len(distances)

In [11]:
model = cp_model.CpModel()

In [12]:
# Step 1: Define Variables
# product_to_slot[p] = slot index where product p is placed
product_to_slot = [model.NewIntVar(0, num_slots - 1, f'product_{p}_slot') for p in range(num_products)]

In [13]:
# Step 2: Each slot holds at most one product
model.AddAllDifferent(product_to_slot)

In [14]:
# Step 3: Define cost expression: sum of (distance of slot * frequency of product)
total_cost = model.NewIntVar(0, 1000, 'total_cost')
cost_terms = []
for p in range(num_products):
    # Add an intermediate variable: distance_of_assigned_slot[p]
    slot_distance = model.NewIntVar(0, max(distances), f'distance_of_product_{p}')
    # Create table: product_to_slot[p] => distances
    model.AddElement(product_to_slot[p], distances, slot_distance)
    # Multiply frequency × distance
    cost_term = model.NewIntVar(0, 1000, f'cost_p{p}')
    model.AddMultiplicationEquality(cost_term, [frequencies[p], slot_distance])
    cost_terms.append(cost_term)

# Total cost = sum of all cost_terms
model.Add(total_cost == sum(cost_terms))
model.Minimize(total_cost)

In [15]:
# Step 4: Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [16]:
# Step 5: Output
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'\nTotal Walking Cost: {solver.Value(total_cost)}')
    for p in range(num_products):
        assigned_slot = solver.Value(product_to_slot[p])
        print(f'Product {p+1} assigned to Slot {assigned_slot+1} (Distance = {distances[assigned_slot]})')
else:
    print("No valid assignment found.")


Total Walking Cost: 74
Product 1 assigned to Slot 2 (Distance = 2)
Product 2 assigned to Slot 3 (Distance = 3)
Product 3 assigned to Slot 1 (Distance = 1)
